<a href="https://colab.research.google.com/github/deltorobarba/machinelearning/blob/master/analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Higher Analysis**

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

2.2.0


## **Analysis**

#### **Glatte Funktionen**

* Eine [glatte Funktion](https://de.m.wikipedia.org/wiki/Glatte_Funktion) ist eine mathematische Funktion, die unendlich oft differenzierbar (insbesondere stetig) ist

* Zum Beispiel ist jede [holomorphe Funktion](https://de.m.wikipedia.org/wiki/Holomorphe_Funktion) (=in jedem Punkt komplex differenzierbar) auch eine glatte Funktion. Außerdem werden glatte Funktionen als Abschneidefunktionen oder als Testfunktionen für Distributionen verwendet.

* Eine Funktion $f: D \rightarrow \mathbb{R}$ heißt unendlich oft (stetig) differenzierbar oder glatt, wenn $f \in C^{n}(D)$ für alle $n \in \mathbb{N}$ gilt. Die Menge aller glatten Funktionen auf $D$ wird mit $C^{\infty}(D)$ notiert und es gilt
$
C^{\infty}(D):=\bigcap_{n \in \mathbb{N}} C^{n}(D)
$

#### **Gewohnliche & Partielle Differentialgleichung**

https://de.m.wikipedia.org/wiki/Differentialgleichung

https://de.m.wikipedia.org/wiki/Gewöhnliche_Differentialgleichung

https://de.wikipedia.org/wiki/Partielle_Differentialgleichung

#### **Funktionenraum**

> **Vektorräume, deren Elemente selbst Funktionen sind, nennt man Funktionenräume.**

A function space is a set of functions between two fixed sets. Often, the domain and/or codomain will have additional structure which is inherited by the function space. For example, the set of functions from any set X into a vector space has a natural vector space structure given by pointwise addition and scalar multiplication. In other scenarios, the function space might inherit a topological or metric structure, hence the name function space.

* Ein [Funktionenraum](https://de.wikipedia.org/wiki/Funktionenraum) ist **eine Menge von Funktionen, die alle denselben Definitionsbereich besitzen**. Allerdings kann der Begriff Funktionenraum ähnlich wie der mathematische Begriff Raum nicht scharf abgegrenzt werden.

* Meist ist ein Funktionenraum mit einer **Vektoraddition und Skalarmultiplikation** versehen, so dass er einen Vektorraum bildet, dann spricht man von einem **linearen Funktionenraum**

* Viele wichtige lineare Funktionenräume sind unendlichdimensional. Diese bilden einen wichtigen Untersuchungsgegenstand der Funktionalanalysis. 

* Lineare Funktionenräume werden häufig mit einer Norm versehen, sodass ein normierter Raum oder – im Falle der Vollständigkeit – sogar ein Banachraum entsteht. In anderen Fällen werden lineare Funktionenräume durch Definition einer Topologie zu einem topologischen Vektorraum oder einem lokalkonvexen Raum.

* Funktionenräume sind im Bereich der linearen Algebra Vektorräume, deren Elemente als Funktionen aufgefasst werden. Hauptsächlich werden Funktionenräume allerdings im Bereich der Funktionalanalysis betrachtet. **Hier wird unter einem Funktionenraum ein Vektorraum mit einer topologischen Struktur verstanden, dessen Elemente als Funktionen aufgefasst werden.**



Meist ist ein Funktionenraum mit einer Vektoraddition und Skalarmultiplikation versehen, so dass er einen Vektorraum bildet, dann spricht man von einem linearen Funktionenraum

* L∞ is a **function space** (Funktionenraum). Its elements are the essentially bounded measurable functions. 

* More precisely, L∞ is defined based on an underlying measure space, (S, Σ, μ). Start with the set of all measurable functions from S to R which are essentially bounded, i.e. bounded up to a set of measure zero. Two such functions are identified if they are equal almost everywhere. Denote the resulting set by L∞(S, μ).

* **The sequence space (Folgenraum) is a special case of the [function space](https://en.m.wikipedia.org/wiki/Function_space) / [Funktionenraum](https://de.m.wikipedia.org/wiki/Funktionenraum): $\ell_{\infty}=L_{\infty}(\mathbb{N})$ where the natural numbers are equipped with the counting measure.** [source](https://en.m.wikipedia.org/wiki/L-infinity)


**In der linearen Algebra**: Sei $D$ eine nichtleere Menge und $V$ ein Vektorraum über einem Körper $K$, dann bezeichnet $V^{D}$ (auch $\mathrm{Abb}(D, V)$ oder $F(D, V)$ ) die Menge aller Funktionen von $D$ nach $V$. Die Menge $V^{D}$ wird für $f, g \in V^{D}$ und für Skalare $\lambda \in K$ durch die folgenden beiden Verknüpfungen zu einem Vektorraum:
Addition: $(f+g): D \rightarrow V, x \mapsto f(x)+g(x)$
Skalarmultiplikation $\lambda f: D \rightarrow V, x \mapsto \lambda \cdot f(x)$
Dieser Vektorraum $V^{D}$ und die Untervektorrăume von $V^{D}$ werden im Bereich der linearen Algebra als Funktionenraum bezeichnet.


**In der Funktionalanalysis:** 

* Sei $D$ eine nichtleere Menge, $V$ ein topologischer Vektorraum (oftmals ein Banachraum oder lokalkonvexer Vektorraum) und $V^{D}$ der Vektorraum aller Abbildungen von $D$ nach $V$. Ein linearer Funktionenraum im Bereich der Funktionalanalysis ist ein Untervektorraum von $V^{D},$ der mit einer von $V$ abgeleiteten topologischen Struktur versehen ist.

* **$C^{p}(D)$ der Raum der $p$ -fach stetig differenzierbaren Funktionen mit $p \in \mathbb{N} \cup\{0, \infty\}$ ist ein Funktionenraum**. **Falls $D$ kompakt ist**, ist der Raum bezüglich der üblichen Norm $\|f\|_{\mathcal{C}^{p}(D)}=\sup _{k \leq p} \sup _{x \in D}\left|f^{(k)}(x)\right|$ **ein Banachraum**. ${ }^{[5]}$ Siehe Differentiationsklasse.

* **$C_{0}^{\infty}, C_{c}^{\infty}$ oder $\mathcal{D}(D)$ der Raum der [Testfunktionen](https://de.wikipedia.org/wiki/Testfunktion) ist ein Funktionenraum**. Er enthält alle glatten Funktionen mit kompaktem Träger und ist mit der Topologie versehen, welche durch den Konvergenzbegriff induziert wird. Eine Folge $\left(\phi_{j}\right)_{j \in J}$ konvergiert in $\mathcal{D}(D)$ gegen $\phi,$ wenn es ein Kompaktum $K \subset D$ gibt mit $\operatorname{supp}\left(\phi_{j}\right) \subset K$ für alle $j,$ und
$\lim _{j \rightarrow \infty} \sup _{x \in K}\left|\partial_{x}^{\alpha}\left(\phi_{j}(x)-\phi(x)\right)\right|=0
$ für alle Multiindizes $\alpha \in \mathbb{N}^{n}$ gilt.

* **$L^{p}(D)$ der Raum der $p$ -fach Lebesgue-integrierbaren Funktionen (siehe $L^{p}$ ) ist ein Funktionenraum & ein zentraler Gegenstand der Funktionalanalysis**. Dieser Raum besteht nicht aus einzelnen Funktionen, sondern aus Äquivalenzklassen von Funktionen, welche sich nur auf einer Lebesgue-Nullmenge unterscheiden. Aus diesem Grund ist für $p \geq 1$ auch die $L^{p}$ -Norm
$ \|f\|_{L^{p}(D)}=\left(\int_{D}|f(x)|^{p} \mathrm{~d} x\right)^{1 / p}
$ positiv definit und damit wirklich eine Norm. Bezüglich dieser Norm ist der $L^{p}$ -Raum auf kompakten Mengen ebenfalls ein Banachraum. Der Spezialfall $L^{2}$ ist sogar ein Hilbertraum. Dieser Raum wird in der Quantenmechanik häufig benutzt. Es ist der Raum der Wellenfunktionen. Für $0<p<1$ kann man die $L^{p}$ -Räume analog definieren, jedoch sind diese keine normierten Räume.

* [Siehe hier mehr](https://de.wikipedia.org/wiki/Funktionenraum#Funktionalanalysis) Beispiele aus der Funktionalanalysis

**In der Topologie**: 

* In der Topologie versteht man unter einem Funktionenraum einen topologischen Raum, dessen Elemente Funktionen von einer Menge oder einem topologischen Raum $X$ in einen topologischen Raum $Y$ sind und dessen Topologie von der Topologie von $X$ und $Y$ und eventuellen Zusatzstrukturen, wie zum Beispiel einer Metrik oder einer uniformen Struktur, abgeleitet ist. Häufig wird die Kompakt-Offen-Topologie verwendet.

* Sind $M$ und $N$ topologische Räume, so schreibt man $\mathcal{C}(M, N)$ für die Menge der stetigen Funktionen $f: M \rightarrow N$.

* Ist auf $N$ eine Metrik $d$ gegeben, dann kann man sinnvoll von der Menge der beschränkten Funktionen sprechen (auch ohne Topologie auf $M$ ). Für diese Abbildungsmenge wird unter anderem die Notation $B(M, N)$ verwendet. Ist auch auf $M$ eine Topologie definiert, schreibt man $\mathcal{C}_{b}(M, N)$ für die Menge der beschränkten stetigen Funktionen. Auf diesen Räumen wird durch
$$
d_{\infty}:(f, g) \mapsto \sup _{x \in M} d(f(x), g(x))
$$
eine Metrik definiert. Alternativ ist auch die Metrik
$$
d_{\infty}^{\prime}:(f, g) \mapsto \min \left\{1, \sup _{x \in M} d(f(x), g(x))\right\}
$$
möglich. **Diese beiden Metriken <u>erzeugen aber dieselben offenen Mengen</u>, sodass sie äquivalent behandelt werden können.**

#### **Folgenraum**

* **Ein [Folgenraum](https://de.m.wikipedia.org/wiki/Folgenraum) (Sequence Space) ist ein in der Mathematik betrachteter Vektorraum**, dessen Elemente Zahlenfolgen sind. 

* Viele in der Funktionalanalysis auftretende Vektorräume sind Folgenräume oder können durch solche repräsentiert werden. Zu den Beispielen zählen u. a. die wichtigen Räume wie ℓ∞ aller beschränkten Folgen oder c0 aller gegen 0 konvergenten Folgen.

* **Werden Normen bzw. Systeme von Normen oder Halbnormen auf Folgenräumen definiert, erhält man normierte Räume bzw. lokalkonvexe Räume.**

* A **sequence space** (Folgenraum) is a vector space whose elements are infinite sequences of real or complex numbers. 

* Equivalently, it is a **function space** (Funktionenraum) whose elements are functions from the natural numbers to the field K of real or complex numbers. 

* The set of all such functions is naturally identified with the set of all possible infinite sequences with elements in K, and **can be turned into a vector space** under the operations of pointwise addition of functions and pointwise scalar multiplication. All sequence spaces are linear subspaces of this space. Sequence spaces are typically equipped with a norm, or at least the structure of a topological vector space.

* The **most important sequence spaces in analysis are the ℓp spaces**, consisting of the p-power summable sequences, with the p-norm. These are special cases of Lp spaces for the counting measure on the set of natural numbers. 

* Other important classes of sequences like convergent sequences or null sequences (Nullfolgen) form sequence spaces, respectively denoted c and c0, with the sup norm. Any sequence space can also be equipped with the topology of pointwise convergence, under which it becomes a special kind of Fréchet space called FK-space.

* The vector space ℓ∞ is a **sequence space** (Folgenraum) whose elements are the bounded sequences. The vector space operations, addition and scalar multiplication, are applied coordinate by coordinate. 

* $\ell^{\infty},$ the (real or complex) vector space of bounded sequences with the **[supremum norm](https://de.m.wikipedia.org/wiki/Supremumsnorm)**, and $L^{\infty}=L^{\infty}(X, \Sigma, \mu)$, the vector space of essentially bounded measurable functions with the **[essential supremum norm](https://de.m.wikipedia.org/wiki/Wesentliches_Supremum)**, are two closely related Banach spaces. 

* In fact the former is a special case of the latter. As a Banach space they are the continuous dual of the Banach spaces $\ell_{1}$ of absolutely summable sequences, and $L^{1}=L^{1}(X, \Sigma, \mu)$ of absolutely integrable measurable functions (if the measure space fulfills the conditions of being localizable and therefore
semifinite). 

* Pointwise multiplication gives them the structure of a Banach algebra, and in fact they are the standard examples of abelian Von Neumann algebras.

#### **Test functions for optimization**

Test functions for optimization (for numerik):

https://towardsdatascience.com/optimization-eye-pleasure-78-benchmark-test-functions-for-single-objective-optimization-92e7ed1d1f12

https://en.m.wikipedia.org/wiki/Test_functions_for_optimization


## **Funktionalanalysis**

* Functional analysis is organized around adequate techniques to bring function spaces as topological vector spaces within reach of the ideas that would apply to normed spaces of finite dimension. 

* Die historischen Wurzeln der Funktionalanalysis liegen im Studium der **Fouriertransformation** und aahnlicher Transformationen sowie der Untersuchung von Differential- und Integralgleichungen. 

* Der Wortbestandteil ”funktional“ geht auf die **Variationsrechnung** zuru ̈ck.

* Grundlage der Funktionalanalysis sind Vektorräume über den reellen oder komplexen Zahlen. Der Grundbegriff ist hier der topologische Vektorraum, der dadurch gekennzeichnet ist, **dass die Vektorraumverknüpfungen stetig sind**, etwas konkreter werden auch lokalkonvexe topologische Vektorräume und Fréchet-Räume untersucht. 

* Wichtige Aussagen sind dabei der Satz von Hahn-Banach, der Satz von Baire und der Satz von Banach-Steinhaus. Insbesondere in der Lösungstheorie partieller Differentialgleichungen spielen diese eine wichtige Rolle, darüber hinaus in der Fredholm-Theorie.

* Der wichtigste Spezialfall **lokalkonvexer topologischer Vektorräume** sind **normierte Vektorräume**. Sind diese zusätzlich vollständig, dann heißen sie **Banachräume**. Noch spezieller betrachtet man **Hilberträume**, bei denen die Norm von einem Skalarprodukt erzeugt wird. Diese Räume sind von grundlegender Bedeutung für die mathematische Formulierung der Quantenmechanik. Ein wichtiger Untersuchungsgegenstand sind stetige lineare Operatoren auf Banach- oder Hilberträumen.



**Definition**

Folgende Themen sollen behandelt werden:

* Distributionen
* Unbeschränkte Operatoren, Spektralsatz 
* Banach-Räume, Hilbert-Räume (insbesondere Lp- und **Sobolev Räume**) und beschränkte lineare Operatoren darauf.
* Pseudodifferentialoperatoren
* Grundlegende Sätze der Funktionalanalysis (Hahn-Banach, Banach-Steinhaus, Satz vom abgeschlossenen Graphen...)
* Kompakte Operatoren und Fredholm Theorie
* Spektraltheorie kompakter Operatoren
* Grundzüge nicht beschränkter Operatoren
* Anwendungen auf Randwertprobleme elliptischer partieller Differentialgleichungen


Der große Reiz der Funktionalanalysis ist, dass die im Grundstudium erworbenen Konzepte der Analysis (Stetigkeit, Grenzwerte, Vollständigkeit) und Linearen Algebra (Vektorräume, lineare Abbildungen, Eigenwerte) konsequent zusammengeführt werden und gemeinsam eine Theorie liefern, die wiederrum in vielen Bereichen der Mathematik und Physik ein unabdingbares Werkzeug ist. Im Rahmen des Masterschwerpunktes Partielle Differentialgleichungen sollen in dieser Vorlesung insbesondere die Anwendung im Studium partieller Differentialgleichungen im Fokus liegen.


**In der Funktionalanalysis betrachtet man topologische Vektorräume, d. h. Vektorräume, auf denen man von Konvergenz sprechen und dementsprechend auch unendliche Summen bilden kann**. Good overview [here](https://matheplanet.com/default3.html?call=article.php?sid=1192&ref=https%3A%2F%2Fwww.google.de%2F)

**Das Ziel der Funktionalanalysis** ist es nicht zuletzt, die **<u>Methoden der mehrdimensionalen Analysis auf unendlichdimensionale Räume auszudehnen und dabei insbesondere Konzepte wie Konvergenz, Stetigkeit und Differenzierbarkeit zu untersuchen</u>**. 

Daher werden a priori **nur Vektorräume betrachtet, die zumindest eine topologische Struktur tragen, also die topologischen Vektorräume**. Zu ihnen zählen unter anderem alle normierten Vektorräume und insbesondere die Banach- und Hilberträume.

#### **Operator**

**Operatoren sind Abbildungen zwischen Funktionenräumen (um sie von den Elementen von Funktionenräumen zu unterscheiden, die Funktionen heissen).** 

* Ahnlich wie bei Funktoren, als strukturerhaltende Abbildung zwischen zwei Kategorien, in denen selbst Funktionen existieren).

* Ein [Operator](https://de.wikipedia.org/wiki/Operator_(Mathematik)) ist eine mathematische Vorschrift, durch die man aus mathematischen Objekten neue Objekte bilden kann.

* Er kann eine standardisierte Funktion oder eine Vorschrift über Funktionen sein. 

* In der Funktionalanalysis hat man es mit Vektorräumen zu tun, deren Elemente selbst Funktionen sind. Diese Räume nennt man auch [Funktionenräume](https://de.wikipedia.org/wiki/Funktionenraum). Um die Elemente (=Funktionen) dieser Vektorräume besser von den Abbildungen zwischen solchen Vektorräumen zu unterscheiden, nennt man letztere auch Operatoren. 

* Daher: **Abbildungen zwischen solchen Vektorräumen von Funktionen / Funktionenräumen bezeichnet man auch als Operatoren.**

* Beispiele für Operatoren sind:

  * Operatoren für die Grundrechenarten: +, -, *, /

  * Funktionaloperatoren (im Sinne der Funktionalanalysis): $\frac{d}{d x} \int \nabla$ grad div rot

![cc](https://raw.githubusercontent.com/deltorobarba/repo/master/operator.png)

[Image Source](https://me-lrt.de/operator-linear-beschrankt)

**Unterschied Operator und Funktion**: 

* Was den mathematischen Formalismus angeht, gibt es keinen Unterschied zwischen einem Operator und einer Funktion (und einer Abbildung). 

* Allerdings gibt es einen 'philosophischen' Unterschied: Operatoren werden meistens für zwei Typen von Funktionen verwendet: a) Grundlegende Rechenoperationen. b) Funktionen, deren Argumente ebenfalls Funktionen sind. c) Funktionen in der Meta-Ebene. Mehr dazu steht im bereits verlinkten Wikipedia-Artikel.

**Beispiele:**

* **Differential**: Der [**Differentialoperator**](https://de.wikipedia.org/wiki/Differentialoperator) $\frac{\mathrm{d}}{\mathrm{d} x}$ zur Bildung von [Differentialen](https://de.wikipedia.org/wiki/Differential_(Mathematik)) (ist eine Funktion, die einer Funktion eine Funktion zuordnet und die Ableitung nach einer oder mehreren Variablen enthält.)

* **Integral**: Der [**Volterraoperator**](https://de.wikipedia.org/wiki/Integraloperator#Volterraoperator) $\int_{0}^{t}$ zur Bildung des [bestimmten Integrals](https://de.wikipedia.org/wiki/Integralrechnung) (ist ein Beispiel fur einen [Integraloperator](https://de.wikipedia.org/wiki/Integraloperator). Operatoren wie diese, die einer Funktion eine Zahl zuordnen, nennt man [Funktional](https://de.wikipedia.org/wiki/Funktional).

  * Die Fourier-Transformation ${\mathcal {F}}$ ist z.B. ein **linearer Operator** (siehe unten). 

* [**Nabla Operator**](https://de.wikipedia.org/wiki/Nabla-Operator) $\nabla$ zur Bestimmung des Gradienten einer mehrdimensionalen Funktion. Mit einem der drei Differentialoperatoren [Gradient](https://de.wikipedia.org/wiki/Gradient_(Mathematik)) (Anwendung im [Gradientenverfahren](https://de.wikipedia.org/wiki/Gradientenverfahren) in der Numerik), [Divergenz](https://de.wikipedia.org/wiki/Divergenz_eines_Vektorfeldes) oder [Rotation](https://de.wikipedia.org/wiki/Rotation_eines_Vektorfeldes) 

* **Spezielle Klassen von Operatoren sind etwa [kompakte Operatoren](https://de.wikipedia.org/wiki/Kompakter_Operator) oder [Fredholm-Operatoren](https://de.wikipedia.org/wiki/Fredholm-Operator).**

**Lineare & nicht-lineare Operatoren**

[Lineare Operatoren](https://de.wikipedia.org/wiki/Linearer_Operator): In Funktionalanalysis betrachtet man Eigenschaften von Abbildungen zwischen (unendlichdimensionalen) Banachräumen. **Lineare Abbildungen heißen [lineare Operatoren](https://de.wikipedia.org/wiki/Lineare_Abbildung)**, nichtlineare Abbildungen werden nichtlineare Operatoren genannt.

* Der Begriff linearer Operator wurde in der Funktionalanalysis (einem Teilgebiet der Mathematik) eingeführt und ist **synonym zum Begriff der linearen Abbildung**. Eine lineare Abbildung ist eine **strukturerhaltende Abbildung zwischen Vektorräumen über einem gemeinsamen Körper**. 

* Werden Vektorräume über dem Körper der reellen oder komplexen Zahlen betrachtet und sind diese mit einer Topologie versehen (lokalkonvexe Räume, normierte Räume, Banachräume), so spricht man vorzugsweise von linearen Operatoren.

* Die Bedeutung linearer Operatoren besteht darin, dass sie die lineare Struktur des unterliegenden Raumes respektieren, d. h., sie sind Homomorphismen zwischen Vektorräumen.

**Anwendungen linearer Operatoren sind:**

* Die Beschreibung von Koordinatentransformationen im dreidimensionalen Euklidischen Raum (**Spiegelung, Drehung, Streckung**) und der [Lorentztransformation](https://de.wikipedia.org/wiki/Lorentz-Transformation) in der vierdimensionalen Raumzeit durch Matrizen.

* Die Darstellung von Observablen in der Quantenmechanik und die Beschreibung der Dynamik eines quantenmechanischen Systems durch seinen Hamilton-Operator H in der Schrödingergleichung.

* Die Entwicklung von Lösungstheorien für Differential- und Integralgleichungen, siehe Sobolew-Raum und Distribution.

#### **Integraloperator**

* Ein [linearer Integraloperator](https://de.wikipedia.org/wiki/Integraloperator) ist ein mathematisches Objekt aus der Funktionalanalysis. Dieses Objekt ist ein linearer Operator, der mit einer bestimmten Integralschreibweise mit einem Integralkern dargestellt werden kann.

* Einige Integraloperatoren nennt man traditionell eher [Integraltransformationen](https://de.wikipedia.org/wiki/Integraloperator#Integraltransformationen). Sie spielen zum Beispiel in der Signalverarbeitung eine wesentliche Rolle und dienen der besseren Handhabe und Analyse des Informationsgehaltes eines Signals x.

  * [Fourier-Transformation](https://de.m.wikipedia.org/wiki/Fourier-Transformation)

  * [Laplace-Transformation](https://de.m.wikipedia.org/wiki/Laplace-Transformation)

#### **(Linear) Functional**

**Function:**
$\mathbb{R} \rightarrow f(x) \rightarrow \mathbb{R}$

**Functional:**
$f(x) \rightarrow J[f(x)] \rightarrow \mathbb{R}$

* **Ein [Funktional](https://de.m.wikipedia.org/wiki/Funktional) bzw. [functional](https://en.m.wikipedia.org/wiki/Functional_(mathematics)) J[y] bildet eine <u>Funktion</u> y(x) auf eine <u>Zahl</u> ab.**

> Sei $V$ ein $\mathbb{K}$ -Vektorraum mit $\mathbb{K} \in\{\mathbb{R}, \mathbb{C}\} .$ Ein Funktional $T$ ist eine Abbildung $T: V \rightarrow \mathbb{K} .$

* eine Funktion, deren Definitionsmenge (Urbilder) als Teilmenge in einem Vektorraum V enthalten ist, während ihre Zielmenge in dem zugehörigen Skalarkörper liegt (zB von einer Fläche R<sup>2</sup> in eine Linie R)

* Exkurs Funktionalanalyse: Hier ist der untersuchte Vektorraum V zumeist ein Funktionenraum, also ein Vektorraum, dessen Elemente reell- oder komplexwertige Funktionen sind, wobei diesen durch Funktionale Skalare zugeordnet werden. 

* Als bedeutendes Beispiel eines solchen Funktionals kann etwa das [Lebesgue-Integral](https://de.m.wikipedia.org/wiki/Lebesgue-Integral) gelten.

**Unterschied zwischen Funktion und Funktional:**

* Eine Funktion bildet Elemente eines Körpers auf andere ab, zum Beispiel. $f: \mathbb{R}^{3} \rightarrow \mathbb{R}^{2}$

* Ein Funktional hingegen ist ein Operator, der Elemente eines Vektorraumes $V$ auf den [Skalarkörper](https://de.wikipedia.org/wiki/Skalar_(Mathematik))
$K$ abbildet, über dem $V$ modelliert ist. Diese Elemente können zum Beispiel selber Funktionen sein
(wenn der Vektorraum ein Funktionenraum ist)

**Example of [Linear functional](https://de.m.wikipedia.org/wiki/Funktional#Lineare_Funktionale)**

* In den meisten Bereichen der Funktionalanalysis, etwa in der Theorie der topologischen Vektorräume, wird der Begriff Funktional (ohne weiteren Zusatz) als Synonym für lineare Funktionale benutzt. 

* Ein solches Funktional ist also definitionsgemäß eine Linearform, also **eine lineare Abbildung des Vektorraumes $V$ in seinen Skalarkörper $K$.**

* Die Menge all dieser Funktionale ist wiederum in natürlicher Form ein Vektorraum uber dem gleichen Körper $\mathbb{K}$, indem man für zwei Funktionale $f$ und $g$ über $V$ die Addition und Skalarmultiplikation punktweise definiert, $d .$ h.

> $
(f+g)(x):=f(x)+g(x) \quad(\lambda f)(x):=\lambda(f(x)), x \in V
$

* Der Vektorraum der linearen Funktionale auf dem Vektorraum $V$ wird der algebraische **Dualraum** genannt und oft mit $V^{*}$ bezeichnet.

**Linearform**

Stetige lineare Funktionale ([Linearform](https://de.m.wikipedia.org/wiki/Linearform)): 

* **In einem topologischen Vektorraum sind im Allgemeinen nicht alle linearen Funktionale steti**g. 

* Die stetigen linearen Funktionale innerhalb des algebraischen Dualraums, also die auf $V$ gegebenen stetigen [Linearformen](https://de.m.wikipedia.org/wiki/Linearform), bilden einen linearen Unterraum von $V^{*}$. **Dies ist der topologische Dualraum von $V$, der in der Funktionalanalysis einer der Hauptgegenstände ist.** 

* Er wird meist mit der Bezeichnung $V'$ gekennzeichnet, von einigen Autoren jedoch auch mit derselben Bezeichnung wie der algebraische Dualraum, also ebenfalls mit $V^{*}$.

* Im Kontext der Funktionalanalysis, das heißt im Falle eines topologischen 
$\mathbb {R}$ - oder $\mathbb {C}$-Vektorraums, sind die betrachteten Linearformen meistens [stetige lineare Funktionale](https://de.m.wikipedia.org/wiki/Funktional#Stetige_lineare_Funktionale).

* Es sei $K$ ein Körper und $V$ ein $K$ -Vektorraum. Eine Abbildung $f: V \rightarrow K$ heißt Linearform, wenn für alle Vektoren $x, y \in V$ und Skalare $\alpha \in K$ gilt:

1. $f(x+y)=f(x)+f(y)$ (**Additivität**);

2. $f(\alpha x)=\alpha f(x)$ (**Homogenität**).

* 'Linear form' is a more modern and abstract concept of 'functional': In modern linear algebra, functional refers to a linear mapping from a vector space $V$ into its field of scalars, i.e., it refers to an element of the dual space $V^{*}$. 

* Die Linearform ist ein Begriff aus dem mathematischen Teilgebiet der linearen Algebra. Man bezeichnet damit eine lineare Abbildung von einem Vektorraum in den zugrundeliegenden Körper.

* **Übergang zu Dualraum**: Die Menge aller Linearformen (= stetigen, linearen Abbildungen) über einem gegebenen Vektorraum $V$ bildet dessen Dualraum $V^{*}$ und damit selbst wieder in natürlicher Weise einen $K$ -Vektorraum.

**[One-form](https://en.m.wikipedia.org/wiki/One-form) (Pfaffsche Form)**

* In linear algebra, a one-form on a vector space is the same as a linear functional (linear form) on the space. 

* The usage of one-form in this context usually distinguishes the one-forms from higher-degree multilinear functionals on the space. For details, see [linear functional / linear form](https://en.m.wikipedia.org/wiki/Linear_form).

#### **Dualraum**

Beim Studium normierter Räume ist die Untersuchung des Dualraumes wichtig. Der Dualraum besteht aus allen **stetigen linearen** Funktionen vom **normierten** Raum in seinen Skalarkörper, also in die reellen oder komplexen Zahlen. Der Bidual, also der Dualraum des Dualraums, muss nicht isomorph zum ursprünglichen Raum sein, aber es gibt stets einen natürlichen Monomorphismus von einem Raum in seinen Bidual. Ist dieser spezielle Monomorphismus auch surjektiv, dann spricht man von einem reflexiven Banachraum.



Dualraum: https://youtu.be/2vvjrBbcTZU

* In den meisten Bereichen der Funktionalanalysis, etwa in der Theorie der [topologischen
Vektorräume](https://de.m.wikipedia.org/wiki/Topologischer_Vektorraum), wird der Begriff Funktional (ohne weiteren Zusatz) als Synonym für lineare Funktionale
benutzt. 

* Ein solches Funktional ist also definitionsgemäß eine [Linearform](https://de.m.wikipedia.org/wiki/Linearform), also eine lineare
Abbildung des Vektorraumes $V$ in seinen Skalarkörper $\mathbb{K}$. 

* Die Menge all dieser Funktionale ist
wiederum in natürlicher Form ein Vektorraum über dem gleichen Körper $\mathbb{K}$, indem man für zwei
Funktionale $f$ und $g$ über $V$ die Addition und Skalarmultiplikation punktweise definiert, d. h.


> $
(f+g)(x):=f(x)+g(x) \quad(\lambda f)(x):=\lambda(f(x)), x \in V
$

Der Vektorraum der linearen Funktionale auf dem Vektorraum $V$ wird der algebraische Dualraum genannt und oft mit $V^{*}$ bezeichnet.

**Beispiele von Dualräumen**

Für den Vektorraum $V=\mathbb{R}^{n}$ **ist der Dualraum kanonisch isomorph zum Vektorraum selbst**, $\mathrm{d} .$ h. $V \cong V$ Der kanonische Isomorphismus $I: \mathbb{R}^{n} \rightarrow\left(\mathbb{R}^{n}\right)^{*}$ wird dabei über das [Standardskalarprodukt](https://de.m.wikipedia.org/wiki/Standardskalarprodukt) vermittelt:

$
(I(x))(y):=\langle x, y\rangle=\sum_{i=1}^{n} x_{i} \cdot y_{i}
$

Für den Vektorraum $V=\mathbb{C}^{n}$ gilt ähnliches wie im ersten Fall, allerdings ist die kanonische Abbildung $I: \mathbb{C}^{n} \rightarrow\left(\mathbb{C}^{n}\right)^{*}$ in diesem Fall semilinear:

$
(I(x))(y):=\langle x, y\rangle=\sum_{i=1}^{n} \overline{x_{i}} \cdot y_{i}
$

Der Dualraum ist in diesem Fall also gleich groB, hat aber bezüglich der kanonischen Abbildung eine
andere Skalarmultiplikation. Im Sinne der linearen Algebra sagt man auch: Der Dualraum ist kanonisch
isomorph zum komplex konjugierten Vektorraum.

*Beispiele topologischer Dualräume*

* Für endlichdimensionale Vektorräume gibt es eine natürliche Topologie ([Normtopologie](https://de.m.wikipedia.org/wiki/Normtopologie)), die aus der euklidischen Norm hervorgeht (genauer gesagt: aus einer beliebigen euklidischen Norm, wenn man eine Basis wählt). Dies ist gerade die Topologie, die der normalen Standard-Analysis zugrunde liegt, und in dieser ist jedes lineare Funktional stetig. Das heißt, der algebraische Dualraum ist gleich dem topologischen Dualraum.

* Im unendlichdimensionalen Fall ist der topologische Dualraum (fast) immer ein echter Teilraum des algebraischen Dualraumes.

* In **normierten Vektorräumen ist ein Funktional $f$ genau dann stetig, wenn es beschränkt ist**, das heißt $\sup _{\|x\| \leq 1}|f(x)|<\infty$. Der topologische Dualraum ist dann automatisch ein Banachraum mit der oben angegebenen Supremumsnorm.

* In Hilberträumen ist der topologische Dualraum kanonisch mit dem Ursprungsraum identifizierbar (Darstellungssatz von Fréchet-Riesz). Die Identifikation erfolgt wie im endlichdimensionalen Fall über das Skalarprodukt: $(I(y)):=\langle\cdot, y\rangle$

* Der topologische Dualraum des Raumes der unendlich oft stetig differenzierbaren Funktionen mit kompaktem Träger auf der reellen Achse (die so genannten Testfunktionen) mit einer bestimmten (hier nicht näher erklärten) Topologie wird als Raum der [Distributionen](https://de.m.wikipedia.org/wiki/Distribution_(Mathematik)) bezeichnet. In diesem Raum liegt auch das weiter oben genannte Beispiel des Dirac-Delta-Funktionals.

#### **Banach- & C*-Algebren**

* Banachalgebren sind mathematische **Objekte der Funktionalanalysis**, die einige wesentliche Eigenschaften mit bekannten Funktionenräumen (z. B. Mengen stetiger oder integrierbarer Funktionen) gemeinsam haben, und deren Verallgemeinerung sie daher sind.

* Eine Banachalgebra ist ein Vektorraum, in dem auch ein Produkt und eine Norm so definiert sind, dass gewisse Verträglichkeitsbedingungen erfüllt sind.

https://mathepedia.de/Banachalgebra.html

https://de.wikipedia.org/wiki/Banachalgebra

https://de.wikipedia.org/wiki/C*-Algebra

#### **Distribution**

https://mathepedia.de/Distributionen.html

* Problem: Losungen mit 'Knicken' in Differentialgleichungen. Begriff einer Ableitung ist zu stark, man muss also die Losungen verallgemeinern.

* zB Ableitung der [Heaviside Funktion](https://de.wikipedia.org/wiki/Heaviside-Funktion) - 

  * Nutzung der Delta Funktion. Delta Funktion ist Null ausserhalb der kritischen Stelle Null.
  * Aber: delta ist keine Funktion im ublichen Sinn. Delta ist Null fast uberall (bez. des Lebesgue Masses)
  * Das heisst jegliches Integral ist Null.

* Losung: Delta wird als 'Distribution' definiert (oder auch: 'verallgemeinerte Funktion'). Funktionen fasst man jetzt als eine Dichte auf (zB Massedichte auf einem eindimensionalen Stab). Delta-Funktion ist dann eine singulare Dichte (Punktmasse).

https://www.youtube.com/watch?v=c5WYrQK_7ls

* Testfunktionen: Miss die Dichte aus mit einem 'virtuellen Messgerat' unterhalb einer Funktion. Sollten stetig und lokalisiert sein. 

* Eine Distribution bezeichnet eine **besondere Art eines Funktionals** (=eine Funktion, die anderen Funktionen eine Zahl zuordnet) im Bereich der partiellen Differentialgleichungen

* Es gibt partielle Differentialgleichungen, die hinreichend oft differenzierbare oder gar keine klassischen Lösungen haben, aber Lösungen im distributionellen Sinn. 

* Daher können Distributionen als eine **Verallgemeinerung von Funktionen** angesehen werden. (z.B. kann man jede Distribution differenzieren.)



Sei $\Omega \subset \mathbb{R}^{n}$ eine offene, nichtleere Menge. Eine Distribution $T$ ist ein stetiges und lineares Funktional auf dem Raum der [Testfunktionen](https://de.m.wikipedia.org/wiki/Testfunktion) $\mathcal{D}(\Omega)$.

In anderen Worten eine Abbildung $T: \mathcal{D}(\Omega) \rightarrow \mathbb{C}$ bzw. $\mathbb{R},$ so dass für alle $\phi_{1}, \phi_{2} \in \mathcal{D}(\Omega)$ und $\lambda \in \mathbb{C}$ gilt $T\left(\phi_{1}+\lambda \phi_{2}\right)=T\left(\phi_{1}\right)+\lambda T\left(\phi_{2}\right)$ und $T\left(\phi_{n}\right) \rightarrow T(\phi)$ wann immer $\phi_{n} \rightarrow \phi$ in $\mathcal{D}(\Omega)$

**Temperierte Distribution**

* Eine [temperierte Distribution](https://de.m.wikipedia.org/wiki/Temperierte_Distribution) ist ein Spezialfall einer Distribution

* Laurent Schwartz führte 1947 den Raum der temperierten Distributionen ein, um die Fourier-Transformation in seine Distributionentheorie integrieren zu können.

**Delta-Distribution** ([Dirac-Funktion](https://de.m.wikipedia.org/wiki/Delta-Distribution))
* ist eine spezielle irreguläre Distribution mit kompaktem Träger. Sie hat in der Mathematik und Physik grundlegende Bedeutung. Ihr übliches Formelsymbol ist δ (kleines Delta).

https://de.m.wikipedia.org/wiki/Distribution_(Mathematik)

**Bump / Test function**

https://www.youtube.com/watch?v=beu28z_q3po

https://en.wikipedia.org/wiki/Bump_function

https://de.m.wikipedia.org/wiki/Testfunktion

https://en.m.wikipedia.org/wiki/Test_functions_for_optimization

## **Variationsrechnung**

* Die [Variationsrechnung](https://de.wikipedia.org/wiki/Variationsrechnung) ist eine **Erweiterung der Funktionalanalysis und beschaeftigt sich mit nichtlinearen Funktionalen** (in der Funktionalanalysis sind es linear Funktionale)

* The [calculus of variations](https://en.m.wikipedia.org/wiki/Calculus_of_variations) is a field that **uses variations, which are small changes in functions and functionals, to find maxima and minima of functionals**: mappings from a set of functions to the real numbers.[a] Functionals are often expressed as definite integrals involving functions and their derivatives. Functions that maximize or minimize functionals may be found using the Euler–Lagrange equation of the calculus of variations.

* In calculus of variations we are **NOT concerned with finding fix points of functions (like local maxima in a function), but rather fix points of functionals.**


* dann führt eine Variation der Wirkung: https://de.m.wikipedia.org/wiki/Feldtheorie_(Physik)#Formalismus

#### **Variational Principle**

* a [variational principle](https://en.m.wikipedia.org/wiki/Variational_principle) is one that enables a problem to be solved using calculus of variations, which concerns finding such functions which optimize the values of quantities that depend upon those functions. 

* For example, the problem of determining the shape of a hanging chain suspended at both ends—a catenary—can be solved using variational calculus, and in this case, the variational principle is the following: The solution is a function that minimizes the gravitational potential energy of the chain.

#### **Brachistochrone & Tautochronie**

Johann I Bernoulli hat sich mit dem **Problem des schnellsten Falles** beschäftigt. Im Jahre 1696 fand er schließlich die Lösung in der **Brachistochrone**. Heute sieht man dies oft als die **Geburtsstunde der Variationsrechnung**.

**Brachistochrone Curve** (in rot): Der Körper gleitet auf einer solchen Bahn schneller zum Ziel als auf jeder anderen Bahn, beispielsweise auf einer geradlinigen, obwohl diese kürzer ist.


![vv](https://upload.wikimedia.org/wikipedia/commons/6/63/Brachistochrone.gif)

**Tautochronie** der Brachistochrone – von jedem Startpunkt auf der Kurve erreichen die Kugeln das „Ziel“ gleichzeitig.

![ff](https://upload.wikimedia.org/wikipedia/commons/b/bd/Tautochrone_curve.gif)

https://en.wikipedia.org/wiki/Brachistochrone_curve

#### **Euler–Lagrange Equation**

* the [Euler equation](https://en.m.wikipedia.org/wiki/Euler–Lagrange_equation) is a **second-order partial differential** equation whose **solutions are the functions for which a given functional is stationary**.

* Because a differentiable functional is stationary at its local extrema, the Euler–Lagrange equation is useful for solving optimization problems in which, given some functional, one seeks the function minimizing or maximizing it. 

* This is analogous to [Fermat's theorem](https://en.m.wikipedia.org/wiki/Fermat%27s_theorem_(stationary_points)) in calculus, stating that at any point where a differentiable function attains a local extremum its derivative is zero.

* In Lagrangian mechanics, according to [Hamilton's principle](https://en.m.wikipedia.org/wiki/Hamilton%27s_principle) of stationary action, the evolution of a physical system is described by the solutions to the Euler equation for the action of the system. In this context Euler equations are usually called Lagrange equations. In classical mechanics, it is equivalent to Newton's laws of motion, but it has the advantage that it takes the same form in any system of generalized coordinates, and it is better suited to generalizations.

  * Hamilton's principle is William Rowan Hamilton's formulation of the [principle of stationary action](https://en.m.wikipedia.org/wiki/Principle_of_least_action) (also called: 'Principle of least action'). It states that the **dynamics of a physical system are determined by a variational problem for a functional based on a single function**, the Lagrangian, which may contain all physical information concerning the system and the forces acting on it. The variational problem is equivalent to and allows for the derivation of the differential equations of motion of the physical system


The Euler–Lagrange equation is an equation satisfied by a function q of a real argument t, which is a stationary point of the functional:

> $S(\boldsymbol{q})=\int_{a}^{b} L(t, \boldsymbol{q}(t), \dot{\boldsymbol{q}}(t)) \mathrm{d} t$

The Euler–Lagrange equation, then, is given by

> $L_{x}(t, q(t), \dot{q}(t))-\frac{\mathrm{d}}{\mathrm{d} t} L_{v}(t, q(t), \dot{q}(t))=0$

* partial derivative of one dimension, then second dimension and then time

#### **Non-linear Functional**

**Example of [Non-linear functional](https://de.m.wikipedia.org/wiki/Funktional#Nichtlineare_Funktionale)**

* Nichtlineare Funktionale traten historisch erstmals in der **Variationsrechnung** auf. 

* Ihr Studium unterscheidet sich grundlegend von dem der oben beschriebenen linearen Funktionale. In der Variationsrechnung setzt man es sich beispielsweise zum Ziel, die Extremalpunkte solcher Funktionalpunkte zu bestimmen. 

* Zu diesem Zweck benötigt man eine Verallgemeinerung des Ableitungsbegriffs der mehrdimensionalen Analysis, d. h. eine Definition des Differentials des Funktionals. In der Variationsrechnung und in den Anwendungen ist dieses Differential unter dem Namen Variationsableitung bekannt, mathematisch präzisiert wird der Begriff z. B. durch die Fréchet-Ableitung und die Gateaux-Ableitung.

#### **Functional Derivative & Integration**

[**Functional derivative**](https://en.m.wikipedia.org/wiki/Functional_derivative)

* Functional derivatives are used in Lagrangian mechanics. They are derivatives of functionals: i.e. they carry information on how a functional changes when the input function changes by a small amount.

[**Functional integration**](https://en.m.wikipedia.org/wiki/Functional_integration)

* Richard Feynman used functional integrals as the central idea in his sum over the histories formulation of quantum mechanics. This usage implies an integral taken over some function space.

#### **Gâteaux-Differential**

* [Gâteaux-Differential](https://de.wikipedia.org/wiki/Gâteaux-Differential) ist eine **Verallgemeinerung des gewöhnlichen Differentiationsbegriffes** dar, indem es die Richtungsableitung auch in unendlichdimensionalen Räumen definiert.

#### **Euler-Lagrange-Gleichung**

https://de.wikipedia.org/wiki/Variationsrechnung#Euler-Lagrange-Gleichung;_Variationsableitung;_weitere_notwendige_bzw._hinreichende_Bedingungen

#### **Fundamentallemma der Variationsrechnung**

https://de.wikipedia.org/wiki/Fundamentallemma_der_Variationsrechnung

#### **Fundamentalsatz der Variationsrechnung**

* Fundamental Theorem of the Calculus of Variations - [Fundamentalsatz der Variationsrechnung](https://de.wikipedia.org/wiki/Fundamentalsatz_der_Variationsrechnung)

* eng verwandt mit dem [weierstraßschen Satz vom Minimum](https://de.wikipedia.org/wiki/Satz_vom_Minimum_und_Maximum)

* Er behandelt die in der Variationsrechnung zentrale Frage, unter welchen Bedingungen reellwertige Funktionale ein Minimum annehmen.

#### **Variation der Elemente**

https://de.wikipedia.org/wiki/Variation_der_Elemente

## **Vektoranalysis**

* Die [Vektoranalysis](https://de.m.wikipedia.org/wiki/Vektoranalysis) ist ein **Teilgebiet der Tensoranalysis**, beschäftigt sich hauptsächlich mit **Vektorfeldern in zwei oder mehr Dimensionen** und verallgemeinert die Differential- und der Integralrechnung.

* Betrachtet werden Vektorfelder, die jedem Punkt des Raumes einen Vektor zuordnen, und [Skalarfelder](https://de.m.wikipedia.org/wiki/Skalarfeld), die jedem Punkt des Raumes einen Skalar zuordnen
  * Skalarfeld $\varphi$: eine Funktion, die jedem Punkt eines Raumes eine reelle Zahl (Skalar) zuordnet, z. B. eine Temperatur, Luftdruck.
  * Wichtige Operationen im Zusammenhang mit Skalarfeldern sind: [Gradient](https://de.m.wikipedia.org/wiki/Gradient_(Mathematik)) eines Skalarfeldes, der ein Vektorfeld ist.
[Richtungsableitung](https://de.m.wikipedia.org/wiki/Richtungsableitung) eines Skalarfeldes. (Eine Verallgemeinerung der Richtungsableitung auf unendlichdimensionale Räume ist das [Gâteaux-Differential](https://de.m.wikipedia.org/wiki/G%C3%A2teaux-Differential).)
  * Ein Skalarfeld ist das einfachste [Tensorfeld](https://de.m.wikipedia.org/wiki/Tensorfeld).

*  Die Temperatur eines Swimmingpools ist ein Skalarfeld: Jedem Punkt wird der Skalarwert seiner Temperatur zugeordnet. Die Wasserbewegung entspricht dagegen einem Vektorfeld, da jedem Punkt ein Geschwindigkeitsvektor zugeordnet wird, der Betrag und Richtung hat.

#### **Randwertproblem**

* [Randwertprobleme](https://de.m.wikipedia.org/wiki/Randwertproblem) nennt man eine wichtige Klasse von Problemstellungen, bei denen zu einer vorgegebenen [Differentialgleichung](https://de.m.wikipedia.org/wiki/Differentialgleichung) Lösungen gesucht werden, die auf dem Rand des Definitionsbereiches vorgegebene Funktionswerte (Randbedingungen) annehmen sollen. 

* Das Gegenstück dazu ist das Anfangswertproblem, bei dem die Lösung für einen beliebigen Punkt im Definitionsbereich vorgegeben wird.

#### **Differentialoperator**

https://www.youtube.com/watch?v=YW-bUVIOpB0&t=51s

* [**Nabla Operator**](https://de.wikipedia.org/wiki/Nabla-Operator) $\nabla$ zur Bestimmung des Gradienten einer mehrdimensionalen Funktion. Mit einem der drei **Differentialoperatoren**.

* Der [**Differentialoperator**](https://de.wikipedia.org/wiki/Differentialoperator) $\frac{\mathrm{d}}{\mathrm{d} x}$ zur Bildung von [Differentialen](https://de.wikipedia.org/wiki/Differential_(Mathematik)) (ist eine Funktion, die einer Funktion eine Funktion zuordnet und die Ableitung nach einer oder mehreren Variablen enthält.)
  * [Gradient](https://de.wikipedia.org/wiki/Gradient_(Mathematik)): Gibt die Richtung und Stärke des steilsten Anstiegs eines Skalarfeldes an. Der Gradient eines Skalarfeldes ist ein Vektorfeld. $\operatorname{grad} \phi:=\vec{\nabla} \phi=\left(\begin{array}{c}\frac{\partial \phi}{\partial x} \\ \frac{\partial \phi}{\partial y} \\ \frac{\partial \phi}{\partial z}\end{array}\right)$
  * [Divergenz](https://de.wikipedia.org/wiki/Divergenz_eines_Vektorfeldes): Gibt die Tendenz eines Vektorfeldes an, von Punkten wegzufließen. $\operatorname{div} \vec{F}:=\vec{\nabla} \cdot \vec{F}=\frac{\partial F_{x}}{\partial x}+\frac{\partial F_{y}}{\partial y}+\frac{\partial F_{z}}{\partial z}$
  * [Rotation](https://de.wikipedia.org/wiki/Rotation_eines_Vektorfeldes): Gibt die Tendenz eines Vektorfeldes an, um Punkte zu rotieren. $\operatorname{rot} \vec{F}:=\vec{\nabla} \times \vec{F}=\left(\begin{array}{c}\frac{\partial F_{z}}{\partial y}-\frac{\partial F_{y}}{\partial z} \\ \frac{\partial F_{x}}{\partial z}-\frac{\partial F_{z}}{\partial x} \\ \frac{\partial F_{y}}{\partial x}-\frac{\partial F_{x}}{\partial y}\end{array}\right)$

* **Diese drei Rechenoperationen sind in der Vektoranalysis von besonderer Bedeutung**, weil sie Felder produzieren, die sich bei räumlicher Drehung des ursprünglichen Feldes mitdrehen. Operativ formuliert: Bei Gradient, Rotation und Divergenz spielt es keine Rolle, ob sie vor oder nach einer Drehung angewendet werden. Diese Eigenschaft folgt aus den **koordinatenunabhängigen** Definitionen. 

**Pseudodifferentialoperatoren**

* Die [Pseudodifferentialoperatoren](https://de.wikipedia.org/wiki/Pseudodifferentialoperator) sind eine Verallgemeinerung von Differentialoperatoren. Sie ermöglichen, aufbauend auf dem Grundbegriff der Distribution und auf der Fouriertransformation, ein ganz neues Verständnis für die Lösung partieller Differen- tialgleichungen.

#### **Integralsätze**

[Integralsatz von Gauß](https://de.m.wikipedia.org/wiki/Gaußscher_Integralsatz)

* Im Folgenden sei das „Integrationsvolumen“ V n-dimensional. Das [Volumenintegral](https://de.m.wikipedia.org/wiki/Volumenintegral) über den Gradienten einer skalaren Größe $\phi$, kann dann in ein [Oberflächenintegral](https://de.m.wikipedia.org/wiki/Oberflächenintegral) (bzw. Hyperflächenintegral) über den Rand dieses Volumens umgewandelt werden:

[Satz von Stokes](https://de.m.wikipedia.org/wiki/Satz_von_Stokes)

* sehr grundlegenden Satz über die Integration von Differentialformen, der den Hauptsatz der Differential- und Integralrechnung erweitert

*  eine Verbindungslinie von der Differentialgeometrie zur Algebraischen Topologie eröffnet. 

* Dieser Zusammenhang wird durch den Satz von de Rham beschrieben, für den der Satz von Stokes grundlegend ist.

#### **Fundamentalzerlegung**

* der [Helmholtzscher Zerlegungssatz](https://de.m.wikipedia.org/wiki/Helmholtz-Theorem) ist der Fundamentalsatz der Vektoranalysis. Beschreibt den allgemeinen Fall. Er sagt aus, dass sich jedes Vektorfeld $\vec{F}$ als eine Überlagerung eines Quellenanteils $\vec{F}_{Q}$ und eines Wirbelanteils $\vec{F}_{W}$ beschreiben lässt.

**[Navier-Cauchy-Gleichungen](https://de.m.wikipedia.org/wiki/Navier-Cauchy-Gleichungen)**

* mathematisches Modell der Bewegung elastischer Festkörper

## **Tensoranalysis**

* Die [Tensoranalysis](https://de.m.wikipedia.org/wiki/Tensoranalysis) ist ein Teilgebiet der Differentialgeometrie beziehungsweise der Differentialtopologie.

* In der Tensoranalysis wird **das Verhalten von geometrischen Differentialoperatoren auf Tensorfeldern untersucht**.

* Sie ist eine **Verallgemeinerung der Vektoranalysis**

* Zum Beispiel kann der Differentialoperator Rotation in diesem Kontext auf n Dimensionen verallgemeinert werden. 

* Zentrale Objekte der Tensoranalysis sind Tensorfelder. Es wird untersucht, wie Differentialoperatoren auf diesen Feldern wirken.

#### **Tensorfeld**

* Tensorfelder sind Funktionen, **die jedem Punkt einen Tensor zuordnen** (Tensor meint in diesem Fall ein rein algebraisches Objekt)

* Tensorfelder werden auf ihre analytischen Eigenschaften untersucht (zB differenziert). Man erhält durch Differenzieren eines Tensorfeldes wieder ein Tensorfeld. Tensorfelder sind besondere glatte Abbildungen, die in Tensorbündel hinein abbilden (siehe unten).

* Sei $M$ eine differenzierbare Mannigfaltigkeit. Ein [Tensorfeld](https://de.m.wikipedia.org/wiki/Tensorfeld) vom Typ (r,s) ist ein glatter [Schnitt](https://de.m.wikipedia.org/wiki/Schnitt_(Faserbündel)) im Tensorbündel $T_{s}^{r}(M)$. 

  * Ein Tensorfeld ist also ein glattes Feld $M \rightarrow T_{s}^{r}(M),$ welches jedem Punkt der Mannigfaltigkeit einen (r,s)-Tensor zuordnet. 

  * Die Menge der Tensorfelder wird oft mit $\Gamma^{\infty}\left(T_{s}^{r}(M)\right)$ bezeichnet.

#### **Tensordichte**

* [Tensordichte](https://de.m.wikipedia.org/wiki/Tensordichte) ist die Quantitätsgröße eines Tensorfeldes

* die Tensordichte ist eine **Verallgemeinerung der Tensorfelder** in der Tensoranalysis 

* wurde eingeführt, um den „Unterschied zwischen Quantität und Intensität, soweit er physikalische Bedeutung hat“, zu erfassen: „die Tensoren sind die Intensitäts-, die Tensordichten die Quantitätsgrößen“. 

* eine **Tensordichte** ordnet einem Koordinatensystem ein Tensorfeld derart zu, dass es bei einem Koordinatenwechsel mit dem Absolutbetrag der Funktionaldeterminante multipliziert wird. Eine Tensordichte der Stufe null ist demnach eine skalare Dichte, deren Integral gemäß dem Transformationssatz eine Invariante liefert.

#### **Vektorbündel & Tensorbündel**

* [Vektorbündel](https://de.m.wikipedia.org/wiki/Vektorbündel) oder manchmal auch Vektorraumbündel sind Familien von Vektorräumen, die **durch die Punkte eines topologischen Raumes parametrisiert sind**.

* Vektorbündel gehören damit auch zu den [Faserbündeln](https://de.m.wikipedia.org/wiki/Faserbündel). Remind: Faser ist ein Urbild von einem Element ("Faser der Abbildung über einem Element") - surjektiv! kann also mehrere Elemente im Urbild haben. Daher Faser $\mathbb{R}$<sup>2</sup> zu Punkt auf $\mathbb{R}$ (siehe [hier](https://de.m.wikipedia.org/wiki/Vektorbündel) die Illustration:


![cc](https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Vectorbundle.svg/320px-Vectorbundle.svg.png)

*Illustration des Vektorbündels $(E, B, \pi)$. Hier ist der **Totalraum** $E=\mathbb{R}^{2}$ und der **Basisraum** $B=\mathbb{R} .$ Die Abbildung $\pi: E \rightarrow B$ projiziert jede Gerade $E_{x}$ auf den Punkt $x$. Der Raum $E_{x}=\{p \in E \mid \pi(p)=x\}$ wird **Faser über $x$** genannt. Außerdem ist der Totalraum $E$ die Vereinigung aller Fasern.* (Comment: also Totalraum E ist Urbild mit Faser und Basisraum B ist Zielbild mit Element das von der Faser stammt)

* [Tensorbündel](https://de.m.wikipedia.org/wiki/Tensoranalysis#Tensorbündel) ist ein bestimmtes Vektorbündel. Tensorfelder sind dann besondere glatte Abbildungen, die in dieses Vektorbündel hinein abbilden.

#### **Schnitt (Faserbündel)**

* [Schnitte](https://de.m.wikipedia.org/wiki/Schnitt_(Faserbündel)) sind Abbildungen, welche in der algebraischen Topologie, insbesondere in der Homotopietheorie, untersucht werden. Insbesondere interessiert man sich dafür, unter welchen Bedingungen solche Abbildungen existieren. 

* Das bekannteste Beispiel von Schnitten sind die [**Differentialformen**](https://de.m.wikipedia.org/wiki/Differentialform).

* Ein Schnitt kann als **Verallgemeinerung des Graphen einer Funktion** aufgefasst werden. 



![cc](https://upload.wikimedia.org/wikipedia/commons/thumb/9/90/Bundle_section.svg/187px-Bundle_section.svg.png)

*Die Abbildung s ist ein Schnitt in einem Faserbündel $p: E \rightarrow B$. Dieser Schnitt s erlaubt es, den Basisraum $B$ mit dem Teilraum $s(B)$ von $E$ zu identifizieren.*

## **Differentialgeometrie**

* Die [Differentialgeometrie](https://de.wikipedia.org/wiki/Differentialgeometrie) stellt die Synthese von Analysis und Geometrie dar.

#### **Vektorfeld**

* In der mehrdimensionalen Analysis und der Differentialgeometrie ist ein [Vektorfeld](https://de.m.wikipedia.org/wiki/Vektorfeld) eine Funktion, die jedem Punkt eines Raumes einen Vektor zuordnet. 

* Das duale Konzept zu einem Vektorfeld ist eine Funktion, die jedem Punkt eine [Linearform](https://de.m.wikipedia.org/wiki/Linearform), [zB [stetige lineare Funktionale](https://de.m.wikipedia.org/wiki/Funktional#Stetige_lineare_Funktionale)] zuordnet, eine solche Abbildung wird [pfaffsche Form](https://de.m.wikipedia.org/wiki/Pfaffsche_Form) (One-Form) genannt.

#### **Differentialformen**

* [Differentialform](https://de.m.wikipedia.org/wiki/Differentialform) erlauben eine koordinatenunabhängige Integration auf allgemeinen orientierten differenzierbaren Mannigfaltigkeiten.

* Beispiele:

  * Glatte Funktionen sind 0-Formen.
  * Pfaffsche Formen sind 1-Formen.

#### **Differenzierbare Mannigfaltigkeit**

* [Differenzierbare Mannigfaltigkeiten](https://de.m.wikipedia.org/wiki/Differenzierbare_Mannigfaltigkeit) sind Objekte in der Differenzialtopologie, und ein Oberbegriff für Kurven, Flächen und andere geometrische Objekte, die – aus der Sicht der Analysis – lokal aussehen wie ein [euklidischer Raum](https://de.m.wikipedia.org/wiki/Euklidischer_Raum).

## **Funktionentheorie**

#### **Holomorphe Funktion**

* [Holomorphe Funktionen](https://de.m.wikipedia.org/wiki/Holomorphe_Funktion) sind an jedem Punkt komplex differenzierbar